# 🧠 SISTEMA EXPERTO: Asistente para Decisión de Cobranza en Consorcios

**Materia:** Desarrollo de Sistemas de IA

**Carrera:** Tecnicatura Superior en Ciencia de Datos e Inteligencia Artificial  

**Cuatrimestre:** 2° - Primer Año  
**Fecha de Entrega:** 02/10/2025

## 👥 INTEGRANTES Y ROLES
- **Anrique Maria:** Documentación *(Frames + Documentación general)*

- **Chino Priscila:** Reglas de Negocio  *(Redes Semánticas + Reglas IF-THEN)*

- **Baccelli Ian:** Motor de Inferencia *(Implementación del Forward Chaining)*

- **Guerrero Sergio:** Especialista en Pruebas *(Casos de Prueba)*

# 1) Introducción al proyecto

**📋 Descripción del Problema**

Los administradores de consorcios deben decidir, para cada propietario, qué acción de cobranza tomar según su estado de deuda (cantidad de expensas vencidas), condición de propietario nuevo, historial de deudas saldadas y datos administrativos como teléfono de emergencia. Estas decisiones suelen ser subjetivas y carentes de uniformidad.

**🎯 Objetivo**

Desarrollar un sistema experto que recomiende acciones de cobranza estandarizadas, coherentes y justificables para cada situación.

**💡 Utilidad**
- Estandarizar criterios de decisión

- Reducir la arbitrariedad

- Proveer trazabilidad sobre por qué se toma cada decisión

**👥 Destinatarios**
- Administradores y asistentes administrativos de consorcios

- Estudiantes y docentes de IA (uso pedagógico)

**🛠️ Qué encontrarán en este Notebook**

- **Representaciones del conocimiento:** Frames (diccionarios) y red semántica

- **Reglas de decisión:** 7 reglas IF-THEN con sistema de prioridades

- **Motor de inferencia**: Encadenamiento hacia adelante (forward chaining)

- **Casos de prueba:** Demostración con diferentes escenarios


**🔍 Estrategia de inferencia elegida**
Encadenamiento hacia adelante por su alineación natural con el flujo operativo: partir de hechos concretos hacia una conclusión, facilitando la transparencia del proceso.

# 2) Representación del conocimiento

Para que nuestro sistema pueda "pensar" como un experto, primero tuvimos que representar su conocimiento de distintas maneras. En esta sección combinamos:

- Frames (Diccionarios): Para organizar todos los datos de cada propietario.

- Red Semántica: Para entender cómo se relacionan los conceptos clave entre sí.

- Reglas SI-ENTONCES: Que son el corazón del sistema, donde capturamos la lógica de decisión paso a paso.


## Frame

Crea un frame (diccionario) con los datos relevantes:  
    - nombre: str  
    - es_nuevo: bool  
    - expensas_vencidas: int (cantidad de períodos impagos)  
    - monto_deuda: float (opcionalmente útil para futuras reglas)  
    - tuvo_deuda_saldada: bool (si tuvo deuda y la canceló recientemente)  
    - telefono_emergencia: str|None (número registrado en la administración)

In [ ]:
# ===============================
# a) Frame simple del propietario
# ===============================
# Usamos una funcion que crea un diccionario con tipos basicos asegurados.

def propietario(nombre, es_nuevo, expensas_vencidas, monto_deuda, tuvo_deuda_saldada, telefono_emergencia):
    return {
        "nombre": str(nombre),
        "tipo": "propietario",
        "es_nuevo": bool(es_nuevo),
        "expensas_vencidas": int(expensas_vencidas),
        "monto_deuda": float(monto_deuda),
        "tuvo_deuda_saldada": bool(tuvo_deuda_saldada),
        "telefono_emergencia": (telefono_emergencia if telefono_emergencia else None),
    }

# Ejemplo minimo (para ver la estructura)
demo = propietario("Demo", False, 0, 0.0, False, None)
print("Estructura del frame:", demo)

Estructura del frame: {'nombre': 'Demo', 'tipo': 'propietario', 'es_nuevo': False, 'expensas_vencidas': 0, 'monto_deuda': 0.0, 'tuvo_deuda_saldada': False, 'telefono_emergencia': None}


## Red semantica.
Además del frame (diccionario) agregamos una red semántica mínima para documentar
relaciones entre conceptos del dominio (Propietario, ExpensasVencidas, Acción, etc.).  
Esta representación no reemplaza a las reglas: las complementa como mapa conceptual
para justificar por qué ciertas acciones responden a determinados niveles o estados.


In [ ]:
# -----------------------------------------------
# Red semántica mínima
# -----------------------------------------------
# Representamos la red como una lista de triples (sujeto, relación, objeto)

# Esto forma nuestro mapa conceptual del dominio
red_semantica = [

    # Relaciones sobre niveles de deuda y sus acciones
    ("Propietario", "puede_tener", "ExpensasVencidas"),
    ("ExpensasVencidas", "nivel", "leve (≤ 1)"),
    ("ExpensasVencidas", "nivel", "moderado (= 2)"),
    ("ExpensasVencidas", "nivel", "severo (= 3)"),
    ("ExpensasVencidas", "nivel", "crítico (≥ 4)"),

    # Estados especiales del propietario
    ("Propietario", "estado", "Nuevo"),
    ("Propietario", "estado", "Saldado"),
    ("Propietario", "debe_aportar", "TelefonoEmergencia"),

    # Relaciones entre niveles y acciones recomendadas
    ("Accion", "responde_a", "nivel leve → Recordatorio amable"),
    ("Accion", "responde_a", "nivel moderado → Notificación formal"),
    ("Accion", "responde_a", "nivel severo → Llamada/WhatsApp"),
    ("Accion", "responde_a", "nivel crítico → Estudio jurídico"),
    ("Accion", "responde_a", "estado Nuevo → Bienvenida + Reglamento interno"),
    ("Accion", "responde_a", "estado Saldado → Carta notificando cierre desde estudio"),
]

# Función para visualizar la red semántica
def mostrar_red(triples):
    # Imprime la red como líneas 'sujeto —relación→ objeto'.
    print("Red semántica (sujeto —relación→ objeto):")
    for s, r, o in triples:
        print(f" - {s} —{r}→ {o}")

#REPRESENTACION EXPLICATIVA DEL RAZONAMIENTO DETRAS DE LA RED SEMANTICA

# Tabla de política por niveles de expensas (coherente con las reglas)
politica_por_nivel = {
    "leve":      "Enviar recordatorio amable por email",
    "moderado":  "Enviar notificación formal",
    "severo":    "Llamada telefónica o WhatsApp para negociar plan de pago",
    "crítico":   "Derivar a estudio jurídico para iniciar acciones legales",
}

def clasificar_nivel_expensas(expensas_vencidas: int) -> str:
    #Clasifica la cantidad de expensas vencidas en niveles
    if expensas_vencidas <= 1 and expensas_vencidas > 0:
        return "leve"
    if expensas_vencidas == 2:
        return "moderado"
    if expensas_vencidas == 3:
        return "severo"
    if expensas_vencidas >= 4:
        return "crítico"
    return "sin_deuda"

def explicar_con_red(p: dict):

    #Usa la red semántica como soporte explicativo (no decisor) para un propietario p.
    nivel = clasificar_nivel_expensas(p["expensas_vencidas"])
    print(f"Propietario: {p['nombre']} / expensas_vencidas={p['expensas_vencidas']} → nivel={nivel}")
    if nivel in politica_por_nivel:
        print("Según la red semántica, la acción asociada al nivel es:")
        print(" -", politica_por_nivel[nivel])
    if p.get("es_nuevo") and p["expensas_vencidas"] == 0 and not p.get("tuvo_deuda_saldada"):
        print("Además, por 'estado Nuevo' → Acción: Enviar carta de bienvenida + Reglamento interno")
    if (p.get("telefono_emergencia") is None) and p["expensas_vencidas"] == 0 and not p.get("tuvo_deuda_saldada"):
        print("Además, falta 'TelefonoEmergencia' → Acción: Solicitar por mail ese dato")

# Mostrar la red y ejemplos explicativos
print("=== MAPA CONCEPTUAL DEL SISTEMA ===")
mostrar_red(red_semantica)

print("\n" + "="*50)
print("EJEMPLOS EXPLICATIVOS (la red no decide, solo explica):")

print("\n--- Ejemplo 1: Carlos (3 expensas vencidas) ---")
ej1 = {"nombre": "Carlos", "expensas_vencidas": 3, "es_nuevo": False,
       "tuvo_deuda_saldada": False, "telefono_emergencia": "11-0000-0000"}
explicar_con_red(ej1)

print("\n--- Ejemplo 2: Laura (propietaria nueva) ---")
ej2 = {"nombre": "Laura", "expensas_vencidas": 0, "es_nuevo": True,
       "tuvo_deuda_saldada": False, "telefono_emergencia": None}
explicar_con_red(ej2)

print("\n--- Ejemplo 3: Roberto (1 expensa vencida) ---")
ej3 = {"nombre": "Roberto", "expensas_vencidas": 1, "es_nuevo": False,
       "tuvo_deuda_saldada": False, "telefono_emergencia": "11-1111-1111"}
explicar_con_red(ej3)

=== MAPA CONCEPTUAL DEL SISTEMA ===
Red semántica (sujeto —relación→ objeto):
 - Propietario —puede_tener→ ExpensasVencidas
 - ExpensasVencidas —nivel→ leve (≤ 1)
 - ExpensasVencidas —nivel→ moderado (= 2)
 - ExpensasVencidas —nivel→ severo (= 3)
 - ExpensasVencidas —nivel→ crítico (≥ 4)
 - Propietario —estado→ Nuevo
 - Propietario —estado→ Saldado
 - Propietario —debe_aportar→ TelefonoEmergencia
 - Accion —responde_a→ nivel leve → Recordatorio amable
 - Accion —responde_a→ nivel moderado → Notificación formal
 - Accion —responde_a→ nivel severo → Llamada/WhatsApp
 - Accion —responde_a→ nivel crítico → Estudio jurídico
 - Accion —responde_a→ estado Nuevo → Bienvenida + Reglamento interno
 - Accion —responde_a→ estado Saldado → Carta notificando cierre desde estudio

EJEMPLOS EXPLICATIVOS (la red no decide, solo explica):

--- Ejemplo 1: Carlos (3 expensas vencidas) ---
Propietario: Carlos / expensas_vencidas=3 → nivel=severo
Según la red semántica, la acción asociada al nivel es:
 - Ll

## Reglas IF–THEN.
Criterios de prioridad (número más bajo = más importante):  
0 = seguimiento post–cancelación (comunicación clave)  
1 = escalamiento legal (4+ expensas)  
2 = 3 expensas (negociación por llamada/WhatsApp)  
3 = 2 expensas (notificación formal)  
4 = hasta 1 expensa (recordatorio)  
5 = falta de teléfono de emergencia (administrativo, no urgente si hay deuda)  
6 = propietario nuevo (bienvenida, lo menos prioritario si hay deuda)

Nota: Si varias reglas aplican, se elige la de menor prioridad.


In [ ]:
# =============================
# Conjunto de reglas de negocio
# =============================
reglas = []

# REGLA PRIORIDAD 0 - Más urgente: comunicación de cierre
reglas.append({
    "id": 5,
    "nombre": "Deuda saldada - notificar cierre desde estudio",
    "si": lambda p: (p["tipo"] == "propietario") and (p["tuvo_deuda_saldada"] is True),
    "entonces": "Enviar carta de parte de la administración notificándole que el estudio jurídico ya le notificó la cancelación de la deuda.",
    "prioridad": 0,  # más importante: cerrar bien el ciclo de cobranza
    "explicacion": "Se comunica formalmente que la deuda fue cancelada; refuerza confianza y cierre del proceso."
})

# REGLA PRIORIDAD 1 - Escalamiento legal
reglas.append({
    "id": 4,
    "nombre": "4+ expensas vencidas - derivar a estudio jurídico",
    "si": lambda p: (p["tipo"] == "propietario") and (p["expensas_vencidas"] >= 4) and (p["tuvo_deuda_saldada"] is False),
    "entonces": "Derivar el caso a un estudio jurídico para iniciar acciones legales",
    "prioridad": 1,
    "explicacion": "Morosidad grave: se escala a instancia legal según política."
})

# REGLA PRIORIDAD 2 - Negociación directa
reglas.append({
    "id": 3,
    "nombre": "3 expensas vencidas - llamada/WhatsApp para negociar",
    "si": lambda p: (p["tipo"] == "propietario") and (p["expensas_vencidas"] == 3) and (p["tuvo_deuda_saldada"] is False),
    "entonces": "Llamada telefónica o mensaje de WhatsApp para negociar un plan de pago",
    "prioridad": 2,
    "explicacion": "Antes de derivar, intentar acuerdo directo por canal inmediato."
})

# REGLA PRIORIDAD 3 - Notificación formal
reglas.append({
    "id": 2,
    "nombre": "2 expensas vencidas - notificación formal",
    "si": lambda p: (p["tipo"] == "propietario") and (p["expensas_vencidas"] == 2) and (p["tuvo_deuda_saldada"] is False),
    "entonces": "Enviar notificación formal",
    "prioridad": 3,
    "explicacion": "Escalamiento formal (comunicación escrita con advertencias)."
})

# REGLA PRIORIDAD 4 - Recordatorio amable
reglas.append({
    "id": 1,
    "nombre": "Hasta 1 expensa vencida - recordatorio amable",
    "si": lambda p: (p["tipo"] == "propietario") and (p["expensas_vencidas"] in (0,1)) and (p["tuvo_deuda_saldada"] is False) and (p["expensas_vencidas"] > 0),
    "entonces": "Enviar recordatorio amable por email",
    "prioridad": 4,
    "explicacion": "Primer nivel de gestión: tono cordial, favorece regularización temprana."
})

# REGLA PRIORIDAD 5 - Telefono de emergencia
reglas.append({
    "id": 7,
    "nombre": "Falta teléfono de emergencia - solicitar por mail",
    "si": lambda p: (p["tipo"] == "propietario") and (p["telefono_emergencia"] is None) and (p["tuvo_deuda_saldada"] is False) and (p["expensas_vencidas"] == 0),
    "entonces": "Enviar notificación por mail solicitando que entregue teléfono de emergencia",
    "prioridad": 5,
    "explicacion": "Tarea administrativa importante, pero no desplaza acciones por deuda."
})

# REGLA PRIORIDAD 6 - Menos urgente: bienvenida
reglas.append({
    "id": 6,
    "nombre": "Propietario nuevo - bienvenida y reglamento interno",
    "si": lambda p: (p["tipo"] == "propietario") and (p["es_nuevo"] is True) and (p["tuvo_deuda_saldada"] is False) and (p["expensas_vencidas"] == 0),
    "entonces": "Enviar carta de bienvenida con el reglamento de interno",
    "prioridad": 6,
    "explicacion": "Acción de onboarding; se posterga si hay deuda u otras urgencias."
})

# Ordenamos por prioridad (ascendente) y luego por id para trazas consistentes
reglas = sorted(reglas, key=lambda r: (r["prioridad"], r["id"]))

# Vista resumida de todas las reglas ordenadas
for r in reglas:
    print(f'[{r["id"]}] prio={r["prioridad"]}: {r["nombre"]} -> {r["entonces"]}')

[5] prio=0: Deuda saldada - notificar cierre desde estudio -> Enviar carta de parte de la administración notificándole que el estudio jurídico ya le notificó la cancelación de la deuda.
[4] prio=1: 4+ expensas vencidas - derivar a estudio jurídico -> Derivar el caso a un estudio jurídico para iniciar acciones legales
[3] prio=2: 3 expensas vencidas - llamada/WhatsApp para negociar -> Llamada telefónica o mensaje de WhatsApp para negociar un plan de pago
[2] prio=3: 2 expensas vencidas - notificación formal -> Enviar notificación formal
[1] prio=4: Hasta 1 expensa vencida - recordatorio amable -> Enviar recordatorio amable por email
[7] prio=5: Falta teléfono de emergencia - solicitar por mail -> Enviar notificación por mail solicitando que entregue teléfono de emergencia
[6] prio=6: Propietario nuevo - bienvenida y reglamento interno -> Enviar carta de bienvenida con el reglamento de interno


# 3) Motor de inferencia (forward).
- Recorre TODAS las reglas y junta las que aplican (`si(...) == True`).  
- Si ninguna aplica -> mensaje por defecto.  
- Si varias aplican -> elige la de mayor prioridad  
- Imprime traza: qué reglas aplicaron y por qué se eligió la final.


In [ ]:
def motor_forward(p, reglas, verbose=True):
    """Aplica encadenamiento hacia adelante sobre un 'propietario'.
    Retorna (accion_elegida, reglas_aplicadas_en_orden).
    """
    aplicadas = []

    if verbose:
        print("\n=== Evaluando propietario:", p["nombre"], "===")
        print("Datos:", p)

    for r in reglas:
        try:
            # Evaluamos condicion: devuelve True/False
            ok = r["si"](p)
        except Exception as e:
            ok = False
            if verbose:
                print(f'  ! Error al evaluar regla [{r["id"]}] {r["nombre"]}:', e)

        if ok:
            aplicadas.append(r)
            if verbose:
                print(f'  ✔ Aplica -> [{r["id"]}] (prio={r["prioridad"]}) {r["nombre"]}')
                print(f'      Acción: {r["entonces"]}')
                print(f'      Por qué: {r["explicacion"]}')

    if not aplicadas:
        accion = "No se pudo determinar una acción (sin reglas coincidentes)"
        if verbose:
            print("  ✗ Ninguna regla aplicó. Acción:", accion)
        return accion, []

    # Resolver conflicto: menor prioridad primero; si empatan, menor id
    aplicadas.sort(key=lambda rr: (rr["prioridad"], rr["id"]))
    elegida = aplicadas[0]

    if verbose:
        print("\n→ ACCIÓN SELECCIONADA:", elegida["entonces"])
        print("   Regla ganadora:", f'[{elegida["id"]}] {elegida["nombre"]} (prio={elegida["prioridad"]})')
        print("   Justificación:", elegida["explicacion"])
        print("=== Fin ===\n")

    return elegida["entonces"], aplicadas

# 4) Demostración (casos de prueba)
Incluye conflictos tipicos para ver la resolución por prioridad y una explicación con la red semántica.


In [ ]:
casos = [
    # Saldo deudas: debe ganar la regla de prioridad 0, aunque sea nuevo, etc.
    propietario("Ana", True, 0, 0.0, True, "11-5555-1111"),
    # 4+ expensas: derivacion legal
    propietario("Bruno", False, 5, 250000.0, False, "11-5555-2222"),
    # 3 expensas: llamada/WhatsApp
    propietario("Carla", False, 3, 90000.0, False, "11-5555-3333"),
    # 2 expensas: notificación formal
    propietario("Diego", False, 2, 50000.0, False, "11-5555-4444"),
    # 1 expensa: recordatorio
    propietario("Elena", False, 1, 15000.0, False, "11-5555-5555"),
    # Sin deuda y sin teléfono de emergencia: pedir datos (gana sobre bienvenida)
    propietario("Fabio", True, 0, 0.0, False, None),
    # Sin deuda, con teléfono, nuevo: bienvenida
    propietario("Gina", True, 0, 0.0, False, "11-5555-6666"),
]

resumen = []
for p in casos:
    accion, aplicadas = motor_forward(p, reglas, verbose=True)
    # Mostrar también como lo explicaria la red semántica (complemento conceptual)
    print("Explicación complementaria (red semántica):")
    explicar_con_red(p)
    resumen.append((p["nombre"], accion))

print("\nResumen final:")
for nombre, acc in resumen:
    print(f" - {nombre}: {acc}")


=== Evaluando propietario: Ana ===
Datos: {'nombre': 'Ana', 'tipo': 'propietario', 'es_nuevo': True, 'expensas_vencidas': 0, 'monto_deuda': 0.0, 'tuvo_deuda_saldada': True, 'telefono_emergencia': '11-5555-1111'}
  ✔ Aplica -> [5] (prio=0) Deuda saldada - notificar cierre desde estudio
      Acción: Enviar carta de parte de la administración notificándole que el estudio jurídico ya le notificó la cancelación de la deuda.
      Por qué: Se comunica formalmente que la deuda fue cancelada; refuerza confianza y cierre del proceso.

→ ACCIÓN SELECCIONADA: Enviar carta de parte de la administración notificándole que el estudio jurídico ya le notificó la cancelación de la deuda.
   Regla ganadora: [5] Deuda saldada - notificar cierre desde estudio (prio=0)
   Justificación: Se comunica formalmente que la deuda fue cancelada; refuerza confianza y cierre del proceso.
=== Fin ===

Explicación complementaria (red semántica):
Propietario: Ana / expensas_vencidas=0 → nivel=sin_deuda

=== Evaluando 

# 5) Evaluacion de tecnicas de representacion.

**Comparacion de tecnicas utilizadas:**

## 📊 Frames (Diccionarios)

**• Rol:** Estructurar y organizar los datos de cada propietario

**• Ventajas:**
  - Sencillos de implementar y entender
  - Fácil validación de tipos de datos
  
**• Limitación:**
  - Solo almacenan datos, no toman decisiones

## 🌐 Red semantica

**• Rol:** Mapa conceptual que conecta niveles de deuda con acciones

**• Ventajas:**
  - Explica el "por qué" detrás de cada decisión
  - Visualiza relaciones entre conceptos clave
  - Facilita la comprensión del dominio

**• Limitación:**
  - Es explicativa, no decisoria
  - No ejecuta acciones por sí misma

## ⚡ Reglas IF-THE + Motor

**• Rol:** Núcleo decisor del sistema

**• Ventajas:**
  - Total transparencia en el razonamiento
  - Fácil auditoría y ajuste de criterios
  - Sistema de prioridades resuelve conflictos

**• Limitación:**
  - Requiere mantenimiento ordenado al escalar
  - Necesita pruebas exhaustivas

## Conclusion

**🎯 MAS EFECTIVA:** **Frames + Reglas IF-THEN**
- Proporciona resultados inmediatos y reproducibles
- Combina estructura de datos con lógica de decisión
- Es práctica y directamente aplicable

**💡 VALOR COMPLEMENTARIO:** **Red Semántica**
- Enriquece la explicabilidad del sistema
- Ideal para documentación y contexto pedagógico
- Ayuda a entender la "filosofía" detrás de las reglas

**📈 VEREDICTO FINAL:** La sinergia entre estas tres técnicas nos permitió construir un sistema que no solo decide eficientemente, sino que también se comunica claramente.

# 6) Fundamentacion del motor de inferencia

**Eleccion:** Encadenamiento hacia adelante

**¿Por que esta eleccion?**

✅ **SE AJUSTA AL FLUJO NATURAL DEL PROBLEMA**
- Parte de datos concretos → llega a acciones específicas
- Responde directamente: "¿Qué hago con este propietario?"

✅ **MAXIMA TRANSPARENCIA**
- Muestra paso a paso cómo se tomó cada decisión
- Explica qué reglas aplicaron y por qué

✅ **EFICIENCIA OPERATIVA**
- Evalúa todas las posibilidades en un solo recorrido
- Resuelve conflictos automáticamente con prioridades

**ALTERNATIVAS DESCARTADAS:**
El encadenamiento hacia atrás era menos apropiado, ya que busca probar hipótesis en lugar de generar acciones inmediatas.

**CONCLUSIÓN:**
Forward Chaining es ideal para nuestro caso: transforma datos en decisiones de manera directa y comprensible.

# 7) Conclusión final del grupo





## 📚 Principales aprendizajes

• **valor de la transparencia**: la capacidad de explicar el "por qué" detrás de cada decisión es tan importante como la decisión misma.

• **combinación efectiva**: la integración de frames para datos, reglas para lógica y red semántica para contexto crea un sistema robusto y comprensible.

• **importancia de prioridades**: un buen sistema de resolución de conflictos es esencial para manejar casos complejos donde aplican múltiples reglas.

## 🚀 Mejoras y ampliaciones futuras

• **historial de deudas**: incorporar un registro histórico para detectar patrones de morosidad y reincidencias.

• **umbrales por monto**: agregar reglas que consideren no solo la cantidad de expensas vencidas, sino también el monto total adeudado.

• **panel administrativo**: desarrollar una interfaz visual para que administradores usen el sistema sin necesidad de programar.


## 💡 Reflexión final

Este proyecto nos permitió transformar teoría en práctica, creando un sistema que no solo toma decisiones inteligentes, sino que también comunica su razonamiento de manera clara - exactamente lo que buscábamos lograr.